In [1]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.3.4'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [670 kB]
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:8 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,401 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 http://security.ubuntu.com/ubunt

In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [7]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

spark.sparkContext.addFile(url)
home_df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), header=True, inferSchema=True)
home_df.show()

+--------------------+-------------------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|               date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+-------------------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08 00:00:00|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13 00:00:00|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12 00:00:00|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16 00:00:00|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08 00:00:00|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa0052

In [10]:
# 2. Create a temporary view of the DataFrame.
home_df.createOrReplaceTempView('home_data')


In [15]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?

average_price_4bed = spark.sql("SELECT ROUND(AVG(price),2), date_built from home_data where bedrooms ==4 GROUP BY date_built").show()

+--------------------+----------+
|round(avg(price), 2)|date_built|
+--------------------+----------+
|           307908.86|      2015|
|           299999.39|      2013|
|           299073.89|      2014|
|           298233.42|      2012|
|           296050.24|      2016|
|           296800.75|      2010|
|            302141.9|      2011|
|           296576.69|      2017|
+--------------------+----------+



In [16]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?

average_price_3bed_3bath = spark.sql("SELECT ROUND(AVG(price),2) AS avg_price, date_built from home_data where bedrooms ==3 AND bathrooms ==3 GROUP BY date_built").show()

+---------+----------+
|avg_price|date_built|
+---------+----------+
| 288770.3|      2015|
|295962.27|      2013|
|290852.27|      2014|
|293683.19|      2012|
|290555.07|      2016|
|292859.62|      2010|
|291117.47|      2011|
|292676.79|      2017|
+---------+----------+



In [17]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?

average_price_3bed_3bath_greater2000sqft = spark.sql("SELECT ROUND(AVG(price),2) AS avg_price, date_built from home_data where bedrooms ==3 AND bathrooms ==3 AND floors =2 AND sqft_living >=2000 GROUP BY date_built").show()

+---------+----------+
|avg_price|date_built|
+---------+----------+
|297609.97|      2015|
|303676.79|      2013|
|298264.72|      2014|
|307539.97|      2012|
| 293965.1|      2016|
|285010.22|      2010|
|276553.81|      2011|
|280317.58|      2017|
+---------+----------+



In [20]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()

view_rating = spark.sql("SELECT view, ROUND(AVG(price),2) AS avg_price FROM home_data WHERE price >=35000 GROUP BY view").show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+----------+
|view| avg_price|
+----+----------+
|  31| 287988.84|
|  85|1056336.74|
|  65| 736679.93|
|  53|  755214.8|
|  78|1080649.37|
|  34| 286124.07|
|  81|1053472.79|
|  28| 285474.25|
|  76|1058802.78|
|  26| 281224.95|
|  27|  281702.6|
|  44| 282856.44|
|  12| 286575.22|
|  91|1137372.73|
|  22| 284908.42|
|  93|1026006.06|
|  47| 292925.62|
|   1| 288248.11|
|  52| 733780.26|
|  13| 285849.95|
+----+----------+
only showing top 20 rows

--- 0.8228998184204102 seconds ---


In [21]:
# 7. Cache the the temporary table home_sales.
spark.sql("CACHE TABLE home_data")

DataFrame[]

In [23]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_data')

True

In [26]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()

view_rating = spark.sql("SELECT view, ROUND(AVG(price),2) AS avg_price FROM home_data WHERE price >=350000 GROUP BY view ORDER BY view").show()


print("--- %s seconds ---" % (time.time() - start_time))


+----+---------+
|view|avg_price|
+----+---------+
|   0|403848.51|
|   1|401044.25|
|   2|397389.25|
|   3| 398867.6|
|   4|399631.89|
|   5|401471.82|
|   6|395655.38|
|   7|403005.77|
|   8|398592.71|
|   9|401393.34|
|  10|401868.43|
|  11|399548.12|
|  12|401501.32|
|  13|398917.98|
|  14|398570.03|
|  15| 404673.3|
|  16|399586.53|
|  17|398474.49|
|  18|399332.91|
|  19|398953.17|
+----+---------+
only showing top 20 rows

--- 0.44951653480529785 seconds ---


In [27]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
home_df.write.partitionBy('date_built').mode("overwrite").parquet('partition_home_data')

In [28]:
# 11. Read the parquet formatted data.
partition_home_df = spark.read.parquet("partition_home_data")

In [31]:
# 12. Create a temporary table for the parquet data.
partition_home_df.createOrReplaceTempView('partition_data')

In [32]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.

start_time = time.time()

view_rating_partition = spark.sql("SELECT view, ROUND(AVG(price),2) AS avg_price FROM partition_data WHERE price >=350000 GROUP BY view ORDER BY view").show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+---------+
|view|avg_price|
+----+---------+
|   0|403848.51|
|   1|401044.25|
|   2|397389.25|
|   3| 398867.6|
|   4|399631.89|
|   5|401471.82|
|   6|395655.38|
|   7|403005.77|
|   8|398592.71|
|   9|401393.34|
|  10|401868.43|
|  11|399548.12|
|  12|401501.32|
|  13|398917.98|
|  14|398570.03|
|  15| 404673.3|
|  16|399586.53|
|  17|398474.49|
|  18|399332.91|
|  19|398953.17|
+----+---------+
only showing top 20 rows

--- 1.0902032852172852 seconds ---


In [34]:
# 14. Uncache the home_sales temporary table.
spark.sql("UNCACHE TABLE home_data")

DataFrame[]

In [35]:
# 15. Check if the home_sales is no longer cached

spark.catalog.isCached('home_data')

False